In [2]:
import numpy as np
import pandas as pd
import neal

In [3]:
data = pd.read_csv('task-1-stocks.csv') 
data.head()

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S90,S91,S92,S93,S94,S95,S96,S97,S98,S99
0,10.829591,21.462513,2.19,0.0007,2.84,79.650002,48.664711,34.250000,108.224541,0.07683,...,35.585823,125.713310,69.806602,51.589684,28.519390,101.637383,56.108028,289.124451,225.103073,178.199997
1,10.648883,20.986433,2.40,0.0006,2.79,76.279999,48.664711,33.299999,106.566994,0.07683,...,33.694645,119.911293,67.153770,52.173275,28.277946,103.727180,54.567417,280.817444,224.172348,174.880005
2,10.584344,21.083593,2.02,0.0009,2.73,75.089996,48.703156,33.049999,105.432350,0.07683,...,33.971889,120.450333,67.037582,52.299721,27.737110,104.549232,55.143902,282.595428,225.053558,173.550003
3,10.519805,20.869841,2.00,0.0007,2.63,73.550003,48.712761,32.400002,105.037697,0.07683,...,34.645187,119.774086,68.121956,52.844410,27.244562,104.707703,55.382450,285.390747,225.449615,173.130005
4,10.584344,20.967001,2.00,0.0011,2.88,76.010002,48.722374,32.599998,105.471817,0.06500,...,34.744202,119.901497,68.828735,52.533154,27.746767,104.162964,56.078205,288.670074,223.964417,176.399994


In [4]:
#data = data.loc[:, (data.min() > 0.01)]
#data.head()

In [5]:
np.array(data.min())

array([1.01958208e+01, 1.88648605e+01, 5.29999971e-01, 5.00000024e-04,
       1.25999999e+00, 6.46399994e+01, 4.86647110e+01, 2.47500000e+01,
       1.05037697e+02, 6.49999976e-02, 1.99999995e-04, 2.61532440e+01,
       1.99999995e-04, 7.69999996e-03, 9.74999964e-02, 1.53999999e-02,
       1.27044609e+02, 9.99999975e-05, 1.34099998e+01, 2.50844307e+01,
       5.16569977e+01, 1.27675247e+02, 1.29300003e+01, 1.64585999e+02,
       1.55415054e+02, 1.33419998e+02, 1.02416649e+02, 7.55500031e+01,
       2.97355927e+02, 4.62829987e+02, 1.81918060e+02, 5.00582047e+01,
       2.28693817e+02, 2.09949997e+02, 6.61451111e+01, 7.45135117e+01,
       1.45925655e+01, 7.49290771e+01, 6.57132797e+01, 1.61336334e+02,
       2.23460281e+02, 9.11900024e+01, 1.07489517e+02, 2.52389999e+02,
       1.41488785e+02, 1.17814262e+02, 1.48193207e+02, 8.55346870e+00,
       1.35320007e+02, 1.59297485e+02, 2.59092926e+02, 3.68100830e+02,
       1.69592239e+02, 1.44570007e+02, 2.29399994e+02, 3.14529999e+02,
      

In [61]:
def build_qubo(data, q, lamb):
    s1 = np.array(data.iloc[0])
    delta =np.array(data.iloc[-1]) - s1
    B = 10**6
    ds = []
    d_sum = 0
    for s1i in s1:
        n_max = int(B/s1i)+1
        d = int(np.log2(n_max))
        ds.append(d)
        d_sum+=(d)
    
    C = np.zeros((data.shape[1], d_sum))
    shift=0
    for i, d in enumerate(ds):
        for j in range(d):
            C[i][shift+j] = 2**j
        shift+=(d)
    
    k = np.ones(data.shape[1])/99
    def dsi(data, i):
        return np.array(data.iloc[i+1]) - np.array(data.iloc[i])
    
    sigma2 = B**(-2) * (98/97 * np.sum([dsi(data,i).T @ dsi(data,i) for i in range(99)]) - 100/(97*98) * delta.T @ delta)
    
    main_d = C.T @ delta/B + 0.08 * q * k * sigma2 @ C + lamb * 2 * C.T * (1/B) @ s1
    main_d = np.diag(main_d)
    
    Q = q* (((k * sigma2 @ C)[:, np.newaxis]) @ [(k * sigma2 @ C)]) + lamb * ((C.T * (1/B) * s1)@(C.T * (1/B) * s1).T)
    Q += main_d
    return -Q, C
Q, C = build_qubo(data, -10, -10) 
Q.shape

(1394, 1394)

In [210]:
#print(( k[:, np.newaxis]).shape, C.shape)
print((((k * sigma2 @ C)[:, np.newaxis]) @ [(k * sigma2 @ C)]).shape)
print(((C.T * (1/B) * s1)@(C.T * (1/B) * s1).T).shape)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 95)

In [58]:
a = np.array([[1,2],[3,4]])
b = np.array([1,2])
b@b
np.linspace(-1.0, 0.0, num=10)

array([-1.        , -0.88888889, -0.77777778, -0.66666667, -0.55555556,
       -0.44444444, -0.33333333, -0.22222222, -0.11111111,  0.        ])

In [59]:
def solve_qubo(data):
    """
    Решаем задачу QUBO.
    """
    best_s = None
    best_return = 0
    sampler = neal.SimulatedAnnealingSampler()
    for a1 in np.linspace(-1.0, 0.0, num=10):
        for a2 in np.linspace(-1.0, 0.0, num=10):
            print(a1,a2)
            Q, C = build_qubo(data, a1, a2)
            response = sampler.sample_qubo(Q, num_reads=2)
            for s in response.samples():
                s = np.asarray(list(s.values()), dtype=np.int32).reshape(-1, 1)
                weights_solution = C @ s
                if weights_solution.sum() >0:
                    average_return, portfolio_values, portfolio_risk = calculate_portfolio_metrics(weights_solution, np.array(data))
                    #print(f"Средняя доходность портфеля: {average_return:.4f}")
                    print(f"{portfolio_values[0][0]:.3f}, {portfolio_values[-1][0]:.3f}, {portfolio_risk:.3f}")
                    #print(f"Риск портфеля: {portfolio_risk:.4f}")
                    if (portfolio_values[0][0] <= 10**6) and portfolio_risk<=0.25 and average_return >= best_return:
                        best_return = average_return
                        best_s = s
    return best_s

In [28]:
def calculate_portfolio_metrics(weights, stock_prices):
    """
    Функция для расчёта метрик инвестиционного портфеля на основе весов акций.
    """
    portfolio_values = np.dot(stock_prices, weights)  
    portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1] 
    average_return = np.mean(portfolio_returns)
    
    n = len(portfolio_returns) 
    risk = np.sqrt(np.sum((portfolio_returns - average_return) ** 2) * n / (n - 1))
    
    return average_return, portfolio_values, risk

In [60]:
Q, C = build_qubo(data, -0.03, -0.19)
solution = solve_qubo(data)
if solution is not None:
    print(solution.reshape(1,-1))
    # Преобразуем решение из словаря в вектор весов
    weights_solution = C @ solution
    
    # Печатаем оптимальные веса
    print("Оптимальные количества для акций:", weights_solution.reshape(1,-1))
    
    # Рассчитаем метрики для найденных оптимальных весов
    average_return, portfolio_values, portfolio_risk = calculate_portfolio_metrics(
        weights_solution, np.array(data))
    
    # Выводим результаты
    print(f"Средняя доходность портфеля: {average_return:.4f}")
    print(f"Текущая стоимость портфеля (в последний день): {portfolio_values[-1][0]} USD")
    print(f"Риск портфеля: {portfolio_risk:.4f}")
else:
    print("err")

-1.0 -1.0
-1.0 -0.8888888888888888
-1.0 -0.7777777777777778
-1.0 -0.6666666666666667
-1.0 -0.5555555555555556
6228.984, 13671.406, 1.979
6055.519, 13290.683, 1.979
-1.0 -0.4444444444444444
157360.849, 330136.368, 1.221
157350.020, 330115.328, 1.221
-1.0 -0.33333333333333337
894320.844, 1772047.860, 0.706
929791.387, 1837336.828, 0.708
-1.0 -0.22222222222222232
1667082.761, 3281864.975, 0.716
1677175.312, 3304016.159, 0.717
-1.0 -0.11111111111111116
5736219.957, 8896270.160, 0.269
5804646.329, 8984528.581, 0.267
-1.0 0.0
38805540.992, 45886529.557, 0.081
38805540.992, 45886529.557, 0.081
-0.8888888888888888 -1.0
-0.8888888888888888 -0.8888888888888888
-0.8888888888888888 -0.7777777777777778
-0.8888888888888888 -0.6666666666666667
-0.8888888888888888 -0.5555555555555556
9840.230, 21597.388, 1.979
8074.027, 17720.917, 1.979
-0.8888888888888888 -0.4444444444444444
173498.090, 365557.196, 1.276
179888.890, 378052.658, 1.243
-0.8888888888888888 -0.33333333333333337
909690.635, 1808091.298, 0